In [39]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

import xgboost as xgb

from sklearn.ensemble import (RandomForestClassifier, AdaBoostClassifier,\
                             GradientBoostingClassifier, ExtraTreesClassifier,\
                             BaggingClassifier, VotingClassifier)

from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Perceptron
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.cross_validation import KFold

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score

from sklearn.metrics import roc_auc_score


from time import time

# Set de entrenamiento y Test final

Cargamos los archivos necesarios

In [40]:
train = pd.read_csv('train_final.csv')


Transformamos las variables categóricas al tipo correspondiente, para reducir el uso de memoria

In [3]:
#train['rango_edad'] = train['rango_edad'].astype('category')
#train['sexo'] = train['sexo'].astype('category') 
#train['nivel_estudios'] = train['nivel_estudios'].astype('category')
#train['esta_estudiando'] = train['esta_estudiando'].astype('category')
#train['tipo_de_trabajo'] = train['tipo_de_trabajo'].astype('category')
#train['nivel_laboral'] = train['nivel_laboral'].astype('category')
#train['nombre_zona'] = train['nombre_zona'].astype('category')
#train['num_area'] = train['num_area'].astype('category')
#train['post_por_nivel'] = train['post_por_nivel'].astype('category')
#train['sepostulo'] = train['sepostulo'].astype('category')

In [4]:
#test_final['rango_edad'] = test_final['rango_edad'].astype('category')
#test_final['sexo'] = test_final['sexo'].astype('category') 
#test_final['nivel_estudios'] = test_final['nivel_estudios'].astype('category')
#test_final['esta_estudiando'] = test_final['esta_estudiando'].astype('category')
#test_final['tipo_de_trabajo'] = test_final['tipo_de_trabajo'].astype('category')
#test_final['nivel_laboral'] = test_final['nivel_laboral'].astype('category')
#test_final['nombre_zona'] = test_final['nombre_zona'].astype('category')
#test_final['num_area'] = test_final['num_area'].astype('category')
#test_final['post_por_nivel'] = test_final['post_por_nivel'].astype('category')

Nos guardamos los IDs necesarios para el submit

In [5]:
#id_aviso_postulante = test_final['id']

Analizamos el set de entrenamiento y el test final

In [41]:
train.sample(5)

,person,brand listing,checkout,conversion,generic listing,searched products,staticpage,viewed product,visited site,label
380,64a3c2f6,14.0,0.0,0.0,1.0,0.0,0.0,13.0,7.0,1.0
705,b866afff,39.0,4.0,2.0,28.0,101.0,2.0,78.0,20.0,1.0
1474,947126c0,0.0,1.0,0.0,2.0,0.0,0.0,2.0,1.0,0.0
160,2b81e8f3,10.0,1.0,1.0,19.0,0.0,0.0,115.0,9.0,1.0
1321,f4924234,1.0,2.0,0.0,0.0,0.0,0.0,20.0,5.0,0.0


In [42]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1980 entries, 0 to 1979
Data columns (total 10 columns):
person               1980 non-null object
brand listing        1980 non-null float64
checkout             1980 non-null float64
conversion           1980 non-null float64
generic listing      1980 non-null float64
searched products    1980 non-null float64
staticpage           1980 non-null float64
viewed product       1980 non-null float64
visited site         1980 non-null float64
label                1980 non-null float64
dtypes: float64(9), object(1)
memory usage: 154.8+ KB


In [28]:
test_final.sample(5)

NameError: name 'test_final' is not defined

In [5]:
test_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19414 entries, 0 to 19413
Data columns (total 10 columns):
person               19414 non-null object
label                19414 non-null int64
brand listing        19414 non-null float64
checkout             19414 non-null float64
conversion           19414 non-null float64
generic listing      19414 non-null float64
searched products    19414 non-null float64
staticpage           19414 non-null float64
viewed product       19414 non-null float64
visited site         19414 non-null float64
dtypes: float64(8), int64(1), object(1)
memory usage: 1.5+ MB


Nos guardamos los features que utilizaremos para los algoritmos de Machine Learning

In [29]:
train.columns

Index(['person', 'brand listing', 'checkout', 'conversion', 'generic listing',
       'searched products', 'staticpage', 'viewed product', 'visited site',
       'label'],
      dtype='object')

In [30]:
# Features
f1 = 'ad campaign hit'
f2 = 'brand listing'
f3 = 'checkout'
f4 = 'conversion'
f5 = 'generic listing'
f6 = 'search engine hit'
f7 = 'viewed product'
f8 = 'visited site'
f9 = 'searched products'
f10= 'staticpage'

features = [f2,f3,f4,f5,f9,f10,f7,f8]

Particionamos el set de entrenamiento para realizar pruebas locales de hiper-parámetros antes de realizar cada submit

In [43]:
test_s = 0.25
random_s = 0

x = np.array(train[features])
y = np.array(train['label'])

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=test_s, random_state=random_s)

Generamos el set de entrenamiento para el test final

In [8]:
x_test_final = np.array(test_final[features])

# Algoritmos probados

# 1. KNN

### Valores iniciales para K

In [44]:
t0 = time()

print('VALORES INICIALES PARA K')

k_valores = [5,10,20,50,60,75,80]
mejor_k = 0
mejor_precision = 0

for k in k_valores:
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(x_train, y_train)
    pred = knn.predict(x_test)
    precision = roc_auc_score(y_test, pred)
    print('La precisión para k=', k, 'es:',precision)
    if precision > mejor_precision:
        mejor_precision = precision
        mejor_k = k
        
print('Con k=', mejor_k, 'se obtuvo la mayor precisión:', mejor_precision)

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

VALORES INICIALES PARA K
La precisión para k= 5 es: 0.6240892606266539
La precisión para k= 10 es: 0.5904695004410755
La precisión para k= 20 es: 0.6194089587349298
La precisión para k= 50 es: 0.609460254190218
La precisión para k= 60 es: 0.6054170614565296
La precisión para k= 75 es: 0.6176528245172672
La precisión para k= 80 es: 0.6150472114222236
Con k= 5 se obtuvo la mayor precisión: 0.6240892606266539
Tiempo de ejecución: 0.07623 seconds.


### Valores grid search para K

In [45]:
t0 = time()

print('VALORES GRID SEARCH PARA K')

k_valores = []

for i in range(mejor_k-2, mejor_k+3):
    k_valores.append(i)
    
for k in k_valores:
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(x_train, y_train)
    pred = knn.predict(x_test)
    precision = roc_auc_score(y_test, pred)
    print('La precisión para k =', k, 'es:',precision)
    if precision > mejor_precision:
        mejor_precision = precision
        mejor_k = k
        
print('Con k=', mejor_k, 'se obtuvo la mayor precisión:', mejor_precision)

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

VALORES GRID SEARCH PARA K
La precisión para k = 3 es: 0.6104159179272716
La precisión para k = 4 es: 0.5819093671382364
La precisión para k = 5 es: 0.6240892606266539
La precisión para k = 6 es: 0.6133482536674617
La precisión para k = 7 es: 0.6245140000653445
Con k= 7 se obtuvo la mayor precisión: 0.6245140000653445
Tiempo de ejecución: 0.03891 seconds.


### Predecimos el test final con el mejor k obtenido

In [11]:
# Creamos el KNN
knn = KNeighborsClassifier(n_neighbors=mejor_k)

# Lo entrenamos con la totalidad del set de datos
knn.fit(x,y)

# Predecimos las postulaciones del set final
pred_final = knn.predict(x_test_final)

### Submit File

In [12]:
#submit = pd.DataFrame({'id':id_aviso_postulante, 'sepostulo':pred_final})

#submit.to_csv('submits/submit_knn.csv', index=False)

#submit['sepostulo'].value_counts()
roc_auc_score(np.array(test_final['label']),pred_final)

0.5030102428299084

# 2. Random Forest

### Set de entrenamiento

In [38]:
t0 = time()

# Parámetros óptimos
params_optimos_rf = {'n_estimators':50, 'max_features':'sqrt', 'max_depth':5, 'min_samples_split':2,\
                     'min_samples_leaf':2, 'bootstrap':True, 'oob_score':True, 'warm_start':True}

# Creamos el Random Forest
random_forest = RandomForestClassifier(**params_optimos_rf)

# Lo entrenamos con nuestro set de entrenamiento
random_forest.fit(x_train,y_train)

# Predecimos valores de nuestro set de datos
pred = random_forest.predict(x_test)

# Evaluamos la precisión
precision = roc_auc_score(y_test, pred)

print("Precisión: ", precision)

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

Precisión:  0.5321377840909091
Tiempo de ejecución: 0.07748 seconds.


### Test final

#### Predicciones con valores binarios

In [13]:
t0 = time()

# Creamos el random forest
random_forest = RandomForestClassifier(**params_optimos_rf)

# Lo entrenamos con la totalidad del set de datos
random_forest.fit(x,y)

# Predecimos las postulaciones del set final
pred_final = random_forest.predict(x_test_final)

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

NameError: name 'x_test_final' is not defined

#### Predicciones con probabilidades

In [15]:
t0 = time()

# Creamos el random forest
random_forest = RandomForestClassifier(**params_optimos_rf)

# Lo entrenamos con la totalidad del set de datos
random_forest.fit(x,y)

# Predecimos las postulaciones del set final
pred_final_proba = random_forest.predict_proba(x_test_final)

# Nos quedamos con la columna correspondiente de probabilidades
df_predicciones = pd.DataFrame(pred_final_proba)
pred_final_proba = np.array(df_predicciones[1])

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

Tiempo de ejecución: 0.19884 seconds.


### Submit files

#### Submit con valores binarios

In [20]:
#submit = pd.DataFrame({'id':id_aviso_postulante, 'sepostulo':pred_final})
#submit.to_csv('submits/submit_random_forest.csv', index=False)

In [16]:
#submit['sepostulo'].value_counts()
roc_auc_score(np.array(test_final['label']),pred_final)

0.5471577032679189

#### Submit con probabilidades

In [22]:
#submit_proba = pd.DataFrame({'id':id_aviso_postulante, 'sepostulo':pred_final_proba})
#submit_proba.to_csv('submits/submit_random_forest_proba.csv', index=False)

In [17]:
#no = submit_proba['sepostulo'] < 0.5
#si = submit_proba['sepostulo'] >= 0.5

#cant_no = submit_proba.loc[(no)].count()
#cant_si = submit_proba.loc[(si)].count()

#print("0   ", cant_no[1])
#print("1   ", cant_si[1])
#print("Name: sepostulo, dtype: int64")
roc_auc_score(np.array(test_final['label']),pred_final_proba)

0.46414566694639225

# 3. Extra Trees

### Set de entrenamiento

In [46]:
t0 = time()

# Parámetros óptimos
params_optimos_et = {'n_estimators':100, 'max_features':'sqrt', 'max_depth':5, 'min_samples_split':2,\
                     'min_samples_leaf':2, 'bootstrap':True, 'oob_score':True, 'warm_start':True}

# Creamos el extra_trees
extra_trees = ExtraTreesClassifier(**params_optimos_et)

# Lo entrenamos con nuestro set de entrenamiento
extra_trees.fit(x_train,y_train)

# Predecimos valores de nuestro set de datos
pred = extra_trees.predict(x_test)

# Evaluamos la precisión
precision = roc_auc_score(y_test, pred)

print("Precisión: ", precision)

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

Precisión:  0.6439948377822067
Tiempo de ejecución: 0.14023 seconds.


### Test final

#### Predicciones con valores binarios

In [19]:
t0 = time()

# Creamos el extra trees
extra_trees = ExtraTreesClassifier(**params_optimos_et)

# Lo entrenamos con la totalidad del set de datos
extra_trees.fit(x,y)

# Predecimos las postulaciones del set final
pred_final = extra_trees.predict(x_test_final)

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

Tiempo de ejecución: 0.36203 seconds.


#### Predicciones con probabilidades

In [20]:
t0 = time()

# Creamos el extra trees
extra_trees = ExtraTreesClassifier(**params_optimos_et)

# Lo entrenamos con la totalidad del set de datos
extra_trees.fit(x,y)

# Predecimos las postulaciones del set final
pred_final_proba = extra_trees.predict_proba(x_test_final)

# Nos quedamos con la columna correspondiente de probabilidades
df_predicciones = pd.DataFrame(pred_final_proba)
pred_final_proba = np.array(df_predicciones[1])

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

Tiempo de ejecución: 0.37548 seconds.


### Submit file

#### Submit con valores binarios

In [27]:
#submit = pd.DataFrame({'id':id_aviso_postulante, 'sepostulo':pred_final})
#submit.to_csv('submits/submit_extra_trees.csv', index=False)

In [21]:
#submit['sepostulo'].value_counts()
roc_auc_score(np.array(test_final['label']),pred_final)

0.47906535837726644

#### Submit con probabilidades

In [22]:
#submit_proba = pd.DataFrame({'id':id_aviso_postulante, 'sepostulo':pred_final_proba})
#submit_proba.to_csv('submits/submit_extra_trees_proba.csv', index=False)
roc_auc_score(np.array(test_final['label']),pred_final_proba)

0.4952222268966175

In [30]:
#no = submit_proba['sepostulo'] < 0.5
#si = submit_proba['sepostulo'] >= 0.5

#cant_no = submit_proba.loc[(no)].count()
#cant_si = submit_proba.loc[(si)].count()

#print("0   ", cant_no[1])
#print("1   ", cant_si[1])
#print("Name: sepostulo, dtype: int64")

# 4. Gaussian Naive Bayes

### Set de entrenamiento

In [47]:
t0 = time()

# Creamos el naive bayes
naive_bayes = GaussianNB()

# Lo entrenamos con nuestro set de entrenamiento
naive_bayes.fit(x_train,y_train)

# Predecimos valores de nuestro set de datos
pred = naive_bayes.predict(x_test)

# Evaluamos la precisión
precision = roc_auc_score(y_test, pred)

print("Precisión: ", precision)

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

Precisión:  0.5437236579867352
Tiempo de ejecución: 0.00285 seconds.


### Test final

#### Predicciones con valores binarios

In [24]:
t0 = time()

# Creamos el naive bayes
naive_bayes = GaussianNB()

# Lo entrenamos con la totalidad del set de datos
naive_bayes.fit(x,y)

# Predecimos las postulaciones del set final
pred_final = naive_bayes.predict(x_test_final)

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

Tiempo de ejecución: 0.00517 seconds.


#### Predicciones con probabilidades

In [25]:
t0 = time()

# Creamos el naive bayes
naive_bayes = GaussianNB()

# Lo entrenamos con la totalidad del set de datos
naive_bayes.fit(x,y)

# Predecimos las postulaciones del set final
pred_final_proba = naive_bayes.predict_proba(x_test_final)

# Nos quedamos con la columna correspondiente de probabilidades
df_predicciones = pd.DataFrame(pred_final_proba)
pred_final_proba = np.array(df_predicciones[1])

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

Tiempo de ejecución: 0.10010 seconds.


### Submit file

#### Submit con valores binarios

In [34]:
#submit = pd.DataFrame({'id':id_aviso_postulante, 'sepostulo':pred_final})
#submit.to_csv('submits/submit_naive_bayes.csv', index=False)

In [26]:
#submit['sepostulo'].value_counts()
roc_auc_score(np.array(test_final['label']),pred_final)

0.3924665602380694

#### Submit con probabilidades

In [27]:
#submit_proba = pd.DataFrame({'id':id_aviso_postulante, 'sepostulo':pred_final_proba})
#submit_proba.to_csv('submits/submit_naive_bayes_proba.csv', index=False)
roc_auc_score(np.array(test_final['label']),pred_final_proba)

0.49787637307282695

In [37]:
#no = submit_proba['sepostulo'] < 0.5
#si = submit_proba['sepostulo'] >= 0.5

#cant_no = submit_proba.loc[(no)].count()
#cant_si = submit_proba.loc[(si)].count()

#print("0   ", cant_no[1])
#print("1   ", cant_si[1])
#print("Name: sepostulo, dtype: int64")

# 5. Perceptron

### Set de entrenamiento

In [48]:
t0 = time()

# Creamos el perceptron
perceptron = Perceptron()

# Lo entrenamos con nuestro set de entrenamiento
perceptron.fit(x_train,y_train)

# Predecimos valores de nuestro set de datos
pred = perceptron.predict(x_test)

# Evaluamos la precisión
precision = roc_auc_score(y_test, pred)

print("Precisión: ", precision)

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

Precisión:  0.5467050021236971
Tiempo de ejecución: 0.00344 seconds.


### Test final

#### Predicciones con valores binarios

In [29]:
t0 = time()

# Creamos el perceptron
perceptron = Perceptron()

# Lo entrenamos con la totalidad del set de datos
perceptron.fit(x,y)

# Predecimos las postulaciones del set final
pred_final = perceptron.predict(x_test_final)

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

Tiempo de ejecución: 0.05273 seconds.


### Submit file

In [40]:
#submit = pd.DataFrame({'id':id_aviso_postulante, 'sepostulo':pred_final})
#submit.to_csv('submits/submit_perceptron.csv', index=False)

In [30]:
#submit['sepostulo'].value_counts()
roc_auc_score(np.array(test_final['label']),pred_final)

0.5572845651225663

# 6. Adaptive Boosting

### Set de entrenamiento

In [49]:
t0 = time()

# Parámetros óptimos
params_optimos_ab = {'n_estimators':50}

# Creamos el adaptive boosting
ada_boost = AdaBoostClassifier(**params_optimos_ab)

# Lo entrenamos con nuestro set de entrenamiento
ada_boost.fit(x_train,y_train)

# Predecimos valores de nuestro set de datos
pred = ada_boost.predict(x_test)

# Evaluamos la precisión
precision = roc_auc_score(y_test, pred)

print("Precisión: ", precision)

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

Precisión:  0.7524095795079558
Tiempo de ejecución: 0.10293 seconds.


### Test final

#### Predicciones con valores binarios

In [32]:
t0 = time()

# Creamos el adaptive boosting
ada_boost = AdaBoostClassifier(**params_optimos_ab)

# Lo entrenamos con la totalidad del set de datos
ada_boost.fit(x,y)

# Predecimos las postulaciones del set final
pred_final = ada_boost.predict(x_test_final)

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

Tiempo de ejecución: 0.00789 seconds.


#### Predicciones con probabilidades

In [33]:
t0 = time()

# Creamos el adaptive boosting
ada_boost = AdaBoostClassifier(**params_optimos_ab)

# Lo entrenamos con la totalidad del set de datos
ada_boost.fit(x,y)

# Predecimos las postulaciones del set final
pred_final_proba = ada_boost.predict_proba(x_test_final)

# Nos quedamos con la columna correspondiente de probabilidades
df_predicciones = pd.DataFrame(pred_final_proba)
pred_final_proba = np.array(df_predicciones[1])

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

Tiempo de ejecución: 0.00770 seconds.


### Submit file

#### Submit con valores binarios

In [45]:
#submit = pd.DataFrame({'id':id_aviso_postulante, 'sepostulo':pred_final})
#submit.to_csv('submits/submit_ada_boost.csv', index=False)

In [34]:
#submit['sepostulo'].value_counts()
roc_auc_score(np.array(test_final['label']),pred_final)

0.5471577032679189

#### Submit con probabilidades

In [35]:
#submit_proba = pd.DataFrame({'id':id_aviso_postulante, 'sepostulo':pred_final_proba})
#submit_proba.to_csv('submits/submit_ada_boost_proba.csv', index=False)
roc_auc_score(np.array(test_final['label']),pred_final_proba)

0.5471577032679189

In [48]:
#no = submit_proba['sepostulo'] < 0.5
#si = submit_proba['sepostulo'] >= 0.5

#cant_no = submit_proba.loc[(no)].count()
#cant_si = submit_proba.loc[(si)].count()

#print("0   ", cant_no[1])
#print("1   ", cant_si[1])
#print("Name: sepostulo, dtype: int64")

# 7. Gradient Boosting

### Set de entrenamiento

In [18]:
t0 = time()

# Parámetros óptimos
params_optimos_gb = {'learning_rate':0.1, 'n_estimators':150, 'max_depth':4, 'min_samples_split':2,\
                     'min_samples_leaf':2, 'subsample':0.5, 'max_features':'sqrt', 'warm_start':True}

# Creamos el gradient boosting
gra_boost = GradientBoostingClassifier(**params_optimos_gb)

# Lo entrenamos con nuestro set de entrenamiento
gra_boost.fit(x_train,y_train)

# Predecimos valores de nuestro set de datos
pred = gra_boost.predict(x_test)

# Evaluamos la precisión
precision = roc_auc_score(y_test, pred)

print("Precisión: ", precision)

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

Precisión:  0.7330430947168949
Tiempo de ejecución: 0.18140 seconds.


### Test final

#### Predicciones con valores binarios

In [37]:
t0 = time()

# Creamos el gradient boosting
gra_boost = GradientBoostingClassifier(**params_optimos_gb)

# Lo entrenamos con la totalidad del set de datos
gra_boost.fit(x,y)

# Predecimos las postulaciones del set final
pred_final = gra_boost.predict(x_test_final)

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

Tiempo de ejecución: 0.45478 seconds.


#### Predicciones con probabilidades

In [38]:
t0 = time()

# Creamos el gradient boosting
gra_boost = GradientBoostingClassifier(**params_optimos_gb)

# Lo entrenamos con la totalidad del set de datos
gra_boost.fit(x,y)

# Predecimos las postulaciones del set final
pred_final_proba = gra_boost.predict_proba(x_test_final)

# Nos quedamos con la columna correspondiente de probabilidades
df_predicciones = pd.DataFrame(pred_final_proba)
pred_final_proba = np.array(df_predicciones[1])

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

Tiempo de ejecución: 0.44393 seconds.


### Submit file

#### Submit con valores binarios

In [39]:
#submit = pd.DataFrame({'id':id_aviso_postulante, 'sepostulo':pred_final})
#submit.to_csv('submits/submit_gra_boost.csv', index=False)
roc_auc_score(np.array(test_final['label']),pred_final)

0.5471577032679189

In [53]:
#submit['sepostulo'].value_counts()

#### Submit con probabilidades

In [40]:
#submit_proba = pd.DataFrame({'id':id_aviso_postulante, 'sepostulo':pred_final_proba})
#submit_proba.to_csv('submits/submit_gra_boost_proba.csv', index=False)
roc_auc_score(np.array(test_final['label']),pred_final_proba)

0.4538110866566438

In [55]:
#no = submit_proba['sepostulo'] < 0.5
#si = submit_proba['sepostulo'] >= 0.5

#cant_no = submit_proba.loc[(no)].count()
#cant_si = submit_proba.loc[(si)].count()

#print("0   ", cant_no[1])
#print("1   ", cant_si[1])
#print("Name: sepostulo, dtype: int64")

# 8. Logistic Regression

### Set de entrenamiento

In [41]:
t0 = time()

# Creamos el logistic regression
log_reg = LogisticRegression()

# Lo entrenamos con nuestro set de entrenamiento
log_reg.fit(x_train,y_train)

# Predecimos valores de nuestro set de datos
pred = log_reg.predict(x_test)

# Evaluamos la precisión
precision = accuracy_score(y_test, pred)

print("Precisión: ", precision)

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

Precisión:  0.9982788296041308
Tiempo de ejecución: 0.05618 seconds.


### Test final

#### Predicciones con valores binarios

In [42]:
t0 = time()

# Creamos el logistic regression
log_reg = LogisticRegression()

# Lo entrenamos con la totalidad del set de datos
log_reg.fit(x,y)

# Predecimos las postulaciones del set final
pred_final = log_reg.predict(x_test_final)

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

Tiempo de ejecución: 0.01917 seconds.


#### Predicciones con probabilidades

In [43]:
t0 = time()

# Creamos el logistic regression
log_reg = LogisticRegression()

# Lo entrenamos con la totalidad del set de datos
log_reg.fit(x,y)

# Predecimos las postulaciones del set final
pred_final_proba = log_reg.predict_proba(x_test_final)

# Nos quedamos con la columna correspondiente de probabilidades
df_predicciones = pd.DataFrame(pred_final_proba)
pred_final_proba = np.array(df_predicciones[1])

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

Tiempo de ejecución: 0.02007 seconds.


### Submit file

#### Submit con valores binarios

In [44]:
#submit = pd.DataFrame({'id':id_aviso_postulante, 'sepostulo':pred_final})
#submit.to_csv('submits/submit_log_reg.csv', index=False)
roc_auc_score(np.array(test_final['label']),pred_final)

0.5218849707616582

In [60]:
#submit['sepostulo'].value_counts()

#### Submit con probabilidades

In [45]:
#submit_proba = pd.DataFrame({'id':id_aviso_postulante, 'sepostulo':pred_final_proba})
#submit_proba.to_csv('submits/submit_log_reg_proba.csv', index=False)
roc_auc_score(np.array(test_final['label']),pred_final_proba)

0.457112218327713

In [62]:
#no = submit_proba['sepostulo'] < 0.5
#si = submit_proba['sepostulo'] >= 0.5

#cant_no = submit_proba.loc[(no)].count()
#cant_si = submit_proba.loc[(si)].count()

#print("0   ", cant_no[1])
#print("1   ", cant_si[1])
#print("Name: sepostulo, dtype: int64")

# 9. Decision Tree

### Set de entrenamiento

In [46]:
t0 = time()

# Parámetros óptimos
params_optimos_dt = {'criterion':'entropy', 'splitter':'best', 'max_depth':None, 'min_samples_split':2,\
                     'min_samples_leaf':2, 'max_features':'sqrt', 'presort':True}

# Creamos el logistic regression
decision_tree = DecisionTreeClassifier(**params_optimos_dt)

# Lo entrenamos con nuestro set de entrenamiento
decision_tree.fit(x_train,y_train)

# Predecimos valores de nuestro set de datos
pred = decision_tree.predict(x_test)

# Evaluamos la precisión
precision = accuracy_score(y_test, pred)

print("Precisión: ", precision)

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

Precisión:  0.9991394148020654
Tiempo de ejecución: 0.00425 seconds.


### Test final

#### Predicciones con valores binarios

In [47]:
t0 = time()

# Creamos el decision tree
decision_tree = DecisionTreeClassifier(**params_optimos_dt)

# Lo entrenamos con la totalidad del set de datos
decision_tree.fit(x,y)

# Predecimos las postulaciones del set final
pred_final = decision_tree.predict(x_test_final)

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

Tiempo de ejecución: 0.00415 seconds.


#### Predicciones con probabilidades

In [48]:
t0 = time()

# Creamos el decision tree
decision_tree = DecisionTreeClassifier(**params_optimos_dt)

# Lo entrenamos con la totalidad del set de datos
decision_tree.fit(x,y)

# Predecimos las postulaciones del set final
pred_final_proba = decision_tree.predict_proba(x_test_final)

# Nos quedamos con la columna correspondiente de probabilidades
df_predicciones = pd.DataFrame(pred_final_proba)
pred_final_proba = np.array(df_predicciones[1])

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

Tiempo de ejecución: 0.00482 seconds.


### Submit file

#### Submit con valores binarios

In [49]:
#submit = pd.DataFrame({'id':id_aviso_postulante, 'sepostulo':pred_final})
#submit.to_csv('submits/submit_decision_tree.csv', index=False)
roc_auc_score(np.array(test_final['label']),pred_final)

0.5471577032679189

In [67]:
#submit['sepostulo'].value_counts()

#### Submit con probabilidades

In [50]:
#submit_proba = pd.DataFrame({'id':id_aviso_postulante, 'sepostulo':pred_final_proba})
#submit_proba.to_csv('submits/submit_decision_tree_proba.csv', index=False)
roc_auc_score(np.array(test_final['label']),pred_final_proba)

0.5471577032679189

In [69]:
#no = submit_proba['sepostulo'] < 0.5
#si = submit_proba['sepostulo'] >= 0.5

#cant_no = submit_proba.loc[(no)].count()
#cant_si = submit_proba.loc[(si)].count()

#print("0   ", cant_no[1])
#print("1   ", cant_si[1])
#print("Name: sepostulo, dtype: int64")

# 10. Bagging

## 10.1 Con Logistic Regression

### Set de entrenamiento

In [51]:
t0 = time()

# Parámetros óptimos
params_optimos_bag_lr = {'base_estimator':LogisticRegression(), 'n_estimators':100, 'bootstrap':True,\
                         'bootstrap_features':True, 'oob_score':True, 'warm_start':False}

# Creamos el bagging
bagging = BaggingClassifier(**params_optimos_bag_lr)

# Lo entrenamos con nuestro set de entrenamiento
bagging.fit(x_train,y_train)

# Predecimos valores de nuestro set de datos
pred = bagging.predict(x_test)

# Evaluamos la precisión
precision = accuracy_score(y_test, pred)

print("Precisión: ", precision)

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

Precisión:  0.9987091222030982
Tiempo de ejecución: 1.31514 seconds.


### Test final

#### Predicciones con valores binarios

In [52]:
t0 = time()

params_optimos_bag_lr = {'base_estimator':LogisticRegression(), 'n_estimators':100, 'bootstrap':True,\
                         'bootstrap_features':True, 'oob_score':True, 'warm_start':False}

# Creamos el bagging
bagging = BaggingClassifier(**params_optimos_bag_lr)

# Lo entrenamos con la totalidad del set de datos
bagging.fit(x,y)

# Predecimos las postulaciones del set final
pred_final = bagging.predict(x_test_final)

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

Tiempo de ejecución: 1.73081 seconds.


#### Predicciones con probabilidades

In [53]:
t0 = time()

params_optimos_bag_lr = {'base_estimator':LogisticRegression(), 'n_estimators':100, 'bootstrap':True,\
                         'bootstrap_features':True, 'oob_score':True, 'warm_start':False}

# Creamos el bagging
bagging = BaggingClassifier(**params_optimos_bag_lr)

# Lo entrenamos con la totalidad del set de datos
bagging.fit(x,y)

# Predecimos las postulaciones del set final
pred_final_proba = bagging.predict_proba(x_test_final)

# Nos quedamos con la columna correspondiente de probabilidades
df_predicciones = pd.DataFrame(pred_final_proba)
pred_final_proba = np.array(df_predicciones[1])

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

Tiempo de ejecución: 1.71528 seconds.


### Submit file

#### Submit con valores binarios

In [54]:
#submit = pd.DataFrame({'id':id_aviso_postulante, 'sepostulo':pred_final})
#submit.to_csv('submits/submit_bagging_log_reg.csv', index=False)
roc_auc_score(np.array(test_final['label']),pred_final)

0.5219441449141228

In [74]:
#submit['sepostulo'].value_counts()

#### Submit con probabilidades

In [55]:
#submit_proba = pd.DataFrame({'id':id_aviso_postulante, 'sepostulo':pred_final_proba})
#submit_proba.to_csv('submits/submit_bagging_log_reg_proba.csv', index=False)
roc_auc_score(np.array(test_final['label']),pred_final_proba)

0.47188856881583063

In [76]:
#no = submit_proba['sepostulo'] < 0.5
#si = submit_proba['sepostulo'] >= 0.5

#cant_no = submit_proba.loc[(no)].count()
#cant_si = submit_proba.loc[(si)].count()

#print("0   ", cant_no[1])
#print("1   ", cant_si[1])
#print("Name: sepostulo, dtype: int64")

## 10.2 Con Decision Tree

### Set de entrenamiento

In [56]:
t0 = time()

# Parámetros óptimos
params_optimos_bag_dt = {'base_estimator':DecisionTreeClassifier(**params_optimos_dt), 'n_estimators':10, 'bootstrap':True,\
                         'bootstrap_features':True, 'oob_score':True, 'warm_start':False}

# Creamos el bagging
bagging = BaggingClassifier(**params_optimos_bag_dt)

# Lo entrenamos con nuestro set de entrenamiento
bagging.fit(x_train, y_train)

# Predecimos valores de nuestro set de datos
pred = bagging.predict(x_test)

# Evaluamos la precisión
precision = accuracy_score(y_test, pred)

print("Precisión: ", precision)

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

Precisión:  0.9569707401032702
Tiempo de ejecución: 0.09848 seconds.


### Test final

#### Predicciones con valores binarios

In [57]:
t0 = time()

# Creamos el bagging
bagging = BaggingClassifier(**params_optimos_bag_dt)

# Lo entrenamos con la totalidad del set de datos
bagging.fit(x,y)

# Predecimos las postulaciones del set final
pred_final = bagging.predict(x_test_final)

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

Tiempo de ejecución: 0.08380 seconds.


#### Predicciones con probabilidades

In [58]:
t0 = time()

# Creamos el bagging
bagging = BaggingClassifier(**params_optimos_bag_dt)

# Lo entrenamos con la totalidad del set de datos
bagging.fit(x,y)

# Predecimos las postulaciones del set final
pred_final_proba = bagging.predict_proba(x_test_final)

# Nos quedamos con la columna correspondiente de probabilidades
df_predicciones = pd.DataFrame(pred_final_proba)
pred_final_proba = np.array(df_predicciones[1])

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

Tiempo de ejecución: 0.09353 seconds.


### Submit file

#### Submit con valores binarios

In [59]:
#submit = pd.DataFrame({'id':id_aviso_postulante, 'sepostulo':pred_final})
#submit.to_csv('submits/submit_bagging_decision_tree.csv', index=False)
roc_auc_score(np.array(test_final['label']),pred_final)

0.5471577032679189

In [81]:
#submit['sepostulo'].value_counts()

#### Submit con probabilidades

In [60]:
#submit_proba = pd.DataFrame({'id':id_aviso_postulante, 'sepostulo':pred_final_proba})
#submit_proba.to_csv('submits/submit_bagging_decision_tree_proba.csv', index=False)
roc_auc_score(np.array(test_final['label']),pred_final_proba)

0.4848502545208166

In [83]:
#no = submit_proba['sepostulo'] < 0.5
#si = submit_proba['sepostulo'] >= 0.5

#cant_no = submit_proba.loc[(no)].count()
#cant_si = submit_proba.loc[(si)].count()

#print("0   ", cant_no[1])
#print("1   ", cant_si[1])
#print("Name: sepostulo, dtype: int64")

# 11. Voting

### Set de entrenamiento

In [61]:
# Creamos los estimadores base que utilizaremos para el voting
estimador_1 = DecisionTreeClassifier(**params_optimos_dt)
estimador_2 = GradientBoostingClassifier(**params_optimos_gb)
estimador_3 = ExtraTreesClassifier(**params_optimos_et)

# Los agregamos a la lista de estimadores
estimadores = []
estimadores.append(('Naive Bayes', estimador_1))
estimadores.append(('Logistic Regression', estimador_2))
estimadores.append(('Random Forest', estimador_3))

# Creamos el voting
voting = VotingClassifier(estimators=estimadores)

# Lo entrenamos con nuestro set de entrenamiento
voting.fit(x_train, y_train)

# Predecimos valores de nuestro set de datos
pred = voting.predict(x_test)

# Evaluamos la precisión
precision = accuracy_score(y_test, pred)

print("Precisión: ", precision)

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

Precisión:  1.0
Tiempo de ejecución: 10.26045 seconds.


/home/mariano/ENTER/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


### Test final

#### Predicciones con valores binarios

In [62]:
t0 = time()

# Creamos el voting
voting = VotingClassifier(estimators=estimadores)

# Lo entrenamos con la totalidad del set de datos
voting.fit(x,y)

# Predecimos las postulaciones del set final
pred_final = voting.predict(x_test_final)

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

Tiempo de ejecución: 0.91246 seconds.


/home/mariano/ENTER/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


### Submit file

#### Submit con valores binarios

In [63]:
#submit = pd.DataFrame({'id':id_aviso_postulante, 'sepostulo':pred_final})
#submit.to_csv('submits/submit_voting.csv', index=False)
roc_auc_score(np.array(test_final['label']),pred_final)

0.5471577032679189

In [87]:
#submit['sepostulo'].value_counts()

# 12. XGBoost

### Set de entrenamiento

In [64]:
t0 = time()

# Creamos el xgboost
xgboost = xgb.XGBClassifier(learning_rate = 0.02,\
                            n_estimators= 2000,\
                            max_depth= 5,\
                            min_child_weight= 2,\
                            gamma=0.9,\                        
                            subsample=0.7,\
                            colsample_bytree=0.7,\
                            objective= 'binary:logistic',\
                            nthread= -1,\
                            scale_pos_weight=1)

# Lo entrenamos con nuestro set de entrenamiento
xgboost.fit(x_train, y_train)

# Predecimos valores de nuestro set de datos
pred_final = xgboost.predict(x_test)

# Evaluamos la precisión
precision = accuracy_score(y_test, pred_final)

print("Precisión: ", precision)

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

SyntaxError: unexpected character after line continuation character (<ipython-input-64-768a22ffce3e>, line 4)

### Test final

#### Predicciones con valores binarios

In [65]:
t0 = time()

# Creamos el xgboost
xgboost = xgb.XGBClassifier(learning_rate = 0.02,\
                            n_estimators= 2000,\
                            max_depth= 5,\
                            min_child_weight= 2,\
                            gamma=0.9,\
                            colsample_bytree=0.7,\
                            objective= 'binary:logistic',\
                            nthread= -1,\
                            scale_pos_weight=1)

# Lo entrenamos con la totalidad del set de datos
xgboost.fit(x,y)

# Predecimos las postulaciones del set final
pred_final = xgboost.predict(x_test_final)

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

Tiempo de ejecución: 1.03294 seconds.


/home/mariano/ENTER/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


#### Predicciones con probabilidades

In [66]:
t0 = time()

# Creamos el xgboost
xgboost = xgb.XGBClassifier(learning_rate = 0.02,\
                            n_estimators= 2000,\
                            max_depth= 5,\
                            min_child_weight= 2,\
                            gamma=0.9,\
                            colsample_bytree=0.7,\
                            objective= 'binary:logistic',\
                            nthread= -1,\
                            scale_pos_weight=1)

# Lo entrenamos con la totalidad del set de datos
xgboost.fit(x,y)

# Predecimos las postulaciones del set final
pred_final_proba = xgboost.predict_proba(x_test_final)

# Nos quedamos con la columna correspondiente de probabilidades
df_predicciones = pd.DataFrame(pred_final_proba)
pred_final_proba = np.array(df_predicciones[1])

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

Tiempo de ejecución: 0.71449 seconds.


### Submit file

#### Submit con valores binarios

In [67]:
#submit = pd.DataFrame({'id':id_aviso_postulante, 'sepostulo':pred_final})
#submit.to_csv('submits/submit_xgboost.csv', index=False)
roc_auc_score(np.array(test_final['label']),pred_final)

0.5471577032679189

In [ ]:
#submit['sepostulo'].value_counts()

#### Submit con probabilidades

In [68]:
#submit_proba = pd.DataFrame({'id':id_aviso_postulante, 'sepostulo':pred_final_proba})
#submit_proba.to_csv('submits/submit_xgboost_proba.csv', index=False)
roc_auc_score(np.array(test_final['label']),pred_final_proba)

0.41126946547307214

In [ ]:
#no = submit_proba['sepostulo'] < 0.5
#si = submit_proba['sepostulo'] >= 0.5

#cant_no = submit_proba.loc[(no)].count()
#cant_si = submit_proba.loc[(si)].count()

#print("0   ", cant_no[1])
#print("1   ", cant_si[1])
#print("Name: sepostulo, dtype: int64")